In [1]:
pip install vosk


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.0 MB 2.1 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/14.0 MB 2.1 MB/s eta 0:00:07
   --- ------------------------------------ 1.3/14.0 MB 2.0 MB/s eta 0:00:07
   ---- ----------------------------------- 1.6/14.0 MB 1.7 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/14.0 MB 1.7 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/14.0 MB 1.7 MB/s eta 0:00:08
   ----- ---------------------------------- 1.8/14.0 MB 1.3 MB/s eta 0:00:10
   ----- ---------------------------------- 1.8/14.0 MB 1.3 MB/s eta 0:00:10
   ----- ---------------------------------- 1.8/14.0 MB 1.3 MB/s eta 0:00:10
   ----- ---------------------------------- 2.1/14.0 MB 903.5 kB

In [ ]:
%pip install pydub
from transformers import pipeline
summarizer = pipeline("summarization")

In [ ]:
from vosk import Model, KaldiRecognizer

FRAME_RATE = 16000
CHANNELS=1

model = Model(model_name="vosk-model-en-us-0.22")
# For a smaller download size, use model = Model(model_name="vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

In [ ]:
import json
from pydub import AudioSegment
mp3 = AudioSegment.from_mp3("marketplace.mp3")
mp3 = mp3.set_channels(CHANNELS)
mp3 = mp3.set_frame_rate(FRAME_RATE)
rec.AcceptWaveform(mp3.raw_data)
result = rec.Result()

text = json.loads(result)["text"]

def voice_recognition(filename):
    model = Model(model_name="vosk-model-small-en-us-0.15")
    rec = KaldiRecognizer(model, FRAME_RATE)
    rec.SetWords(True)

    mp3 = AudioSegment.from_mp3(filename)
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)

    step = 45000
    transcript = ""
    for i in range(0, len(mp3), step):
        print(f"Progress: {i/len(mp3)}")
        segment = mp3[i:i+step]
        rec.AcceptWaveform(segment.raw_data)
        result = rec.Result()
        text = json.loads(result)["text"]
        transcript += text

In [ ]:
transcript = voice_recognition("marketplace.mp3")

In [ ]:
%ffmpeg -i marketplace.mp3 marketplace.wav


In [ ]:
%pip install SpeechRecognition


In [ ]:
import speech_recognition as sr

def voice_recognition(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)  # Capture audio
        try:
            transcript = recognizer.recognize_google(audio)
            return transcript
        except sr.UnknownValueError:
            return "Could not understand the audio"
        except sr.RequestError:
            return "Error with the Speech Recognition API"

transcript = voice_recognition("marketplace.wav")
print(transcript)


In [ ]:
split_tokens = transcript.split(" ")
docs = []
for i in range(0, len(split_tokens), 850):
    selection = " ".join(split_tokens[i:(i+850)])
    docs.append(selection)

In [ ]:
summaries = summarizer(docs)
summary = "\n\n".join([d["summary_text"] for d in summaries])
print(summary)